In [9]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.core import Workspace, Datastore, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, StepSequence
from azureml.core.runconfig import RunConfiguration
from azureml.widgets import RunDetails

ws = Workspace.from_config()
blob_store = Datastore(ws, "workspaceblobstore")    # workspaceblobstore is the default

# Create Experiment
experiment_name = 'mdrift_pipeline'
exp = Experiment(ws, experiment_name)

# Create Compute Target
aml_compute_target = "vdsm-cluster-d13"  # ALL SMALL LETTER, NO underscore, 16ws long only.
aml_compute = AmlCompute(ws, aml_compute_target)

# Use an existing env which was registered to the ws
env = Environment.get(workspace=ws,name="vdsm_all",version="5")

In [11]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.pipeline.steps import PythonScriptStep, ParallelRunStep, ParallelRunConfig
from azureml.pipeline.core.pipeline_output_dataset import PipelineOutputTabularDataset
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.core.graph import PipelineParameter

script_run_config = RunConfiguration()
script_run_config.target = aml_compute
script_run_config.environment = env

veg_type_pipeline_param = PipelineParameter(name="veg_type", default_value='bs')

example_step = PythonScriptStep(
    name="mdrift_pipeline",
    script_name="drift_pipeline_dev.py",
    arguments=[
        "--veg_type", veg_type_pipeline_param
    ],
    runconfig=script_run_config,
    source_directory="."
    )

step_sequence = StepSequence(steps=[example_step])

pipeline = Pipeline(workspace=ws, steps=step_sequence)
pipeline_run = exp.submit(pipeline)
print('Datadrift pipeline submitted for execution')
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()

Created step mdrift_pipeline [7e094293][6370f72f-9fd0-4816-90ef-bc2ef88d511b], (This step will run and generate new outputs)
Submitted PipelineRun 4e6e546f-3c68-40b1-9d20-a4c910626203
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/4e6e546f-3c68-40b1-9d20-a4c910626203?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
Datadrift pipeline submitted for execution
PipelineRunId: 4e6e546f-3c68-40b1-9d20-a4c910626203
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/4e6e546f-3c68-40b1-9d20-a4c910626203?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: a4c75562-a3ed-4098-ae68-ae5da88bbdf8
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/a4c75562-a3ed-4098-ae68-ae5da88bbdf8?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
StepRun( mdrift_pipeline ) Status: Running

StepRun(mdrift_pipeline) Execution Summary
StepRun( mdrift_pipeline ) Status: Finished
{'runId': 'a4c75562-a3ed-4098-ae68-ae5da88bbdf8', 'target': 'vdsm-cluster-d13', 'status': 'Completed', 'startTimeUtc': '2022-02-18T07:39:37.20242Z', 'endTimeUtc': '2022-02-18T07:40:13.266597Z', 'services': {}, 'properties': {'ContentSnapshotId': 'cabedd67-e39a-47ec-9022-8a88587d25a1', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.moduleid': '6370f72f-9fd0-4816-90ef-bc2ef88d511b', 'azureml.moduleName': 'mdrift_pipeline', 'azureml.runsource': 'azureml.StepR

'Finished'

KeyError: 'log_files'